# Create HEPEVT files for radiologicals

Radiological ROOT file is from Juergen and Gleb.

This script is adpated from https://github.com/yuntsebaryon/GAMPix4SNe/blob/main/OldG4Format/G4IO/rootifyRad.ipynb, the one for the old format.

## Units

cm, GeV/c, ns

for both the ROOT and HEPEVT files

## LAr Geometry

X is the drift direction, vertical \
Xmin = -425.0cm \
Xmax = 425.08cm \
Cathode = -325cm \
CRP = 325cm \
Ymin = -753.8024cm \
Ymax = 753.8024cm \
Zmin = -107.0cm \
Zmax = 2198.88cm

Strategy: To reduce the processing time, I decide to futher reduce the LAr geometry.  I fix the location of neutrino interactions in each sample, i.e. the center in the y-z plane, and cut a square 150cm away from the center of the y-z plane.  Therefore it is a 300x300 cm$^2$ in the y-z plane.  The 150cm is obtained from my containment study for the SNS/COHERENT neutrino source; 96% of the total deposited energy from the SNS $\nu_e$s (from $\pi$ decays at rest) will be contained by the cube border 95cm away from the center (190x190x190cm$^3$).

In [1]:
import ROOT

In [2]:
def writeHEPEVT( tRad, prefix, subrun, ymin, ymax, zmin, zmax, nEvents):

    hepevtName = f'{prefix}_{subrun:02d}.hepevt'
    yCenter = ( ymin + ymax )/2.
    zCenter = ( zmin + zmax )/2. - (-107+2198.88)/2
    nParticlePerVtx = 1

    with open(hepevtName, 'w') as f:
        lastEvt = None
        iVertex = 0
        for iPart, iEvt in enumerate( tRad ):
            if ( iEvt.y > ymax or iEvt.y < ymin or iEvt.z > zmax or iEvt.z < zmin ): continue

            if iEvt.event-1 != lastEvt:
                iVertex = 0
            f.write( f'{iEvt.event-1} {iVertex} {nParticlePerVtx}\n')
            
            # ISTHEP IDHEP JMOHEP1 JMOHEP2 JDAHEP1 JDAHEP2 PHEP1 PHEP2 PHEP3 PHEP4 PHEP5 VHEP1 VHEP2 VHEP3 VHEP4
            # final-state particle
            ISTHEP = 1
            IDHEP = iEvt.pdg_code
            # The JMOHEP1, JMOHEP2, JDAHEP1, and JDAHEP2 entries record the indices (between 1 and NHEP, inclusive) 
            # of particles in the event record that correspond to the first mother, second mother, first daughter, 
            # and last daughter of the current particle, respectively. 
            JMOHEP1 = 0
            JMOHEP2 = 0
            JDAHEP1 = 0
            JDAHEP2 = 0

            PHEP1 = iEvt.px
            PHEP2 = iEvt.py
            PHEP3 = iEvt.pz
            PHEP4 = iEvt.Etot
            PHEP5 = iEvt.mass
            VHEP1 = iEvt.x
            VHEP2 = iEvt.y - yCenter
            VHEP3 = iEvt.z - zCenter
            VHEP4 = iEvt.t
            f.write( f'{ISTHEP} {IDHEP} {JMOHEP1} {JMOHEP2} {JDAHEP1} {JDAHEP2} {PHEP1} {PHEP2} {PHEP3} {PHEP4} {PHEP5} {VHEP1} {VHEP2} {VHEP3} {VHEP4}\n')
            iVertex += 1
            lastEvt = iEvt.event-1
    return        

In [3]:
Dir = '/Users/yuntse/data/lartpc_rd/gampix/radiologicals'
infile = f'{Dir}/dune/fullgeoanatruth-vd1x8x14-1000.root'
prefix = f'{Dir}/gen/fullgeoanatruth-vd-reduced'

In [4]:
# Division
nY = 4
nZ = 7
nEvents = 1000

In [5]:
# cm
YLow = -600.
ZLow = 0.
YLength = 300.
ZLength = 300.

In [6]:
fRad = ROOT.TFile( infile )
tRad = fRad.Get("fullgeoanatruth/FullGeoAnaTruth")

In [7]:
for iy in range( nY ):
    for iz in range( nZ ):

        ymin = YLength*float(iy) + YLow
        ymax = YLength*float(iy+1) + YLow
        zmin = ZLength*float(iz) + ZLow
        zmax = ZLength*float(iz+1) + ZLow
        subrun = iy*nZ + iz

        print( f'Processing Subrun {subrun}, {ymin} < y < {ymax}cm, {zmin} < z < {zmax}cm...')
        writeHEPEVT( tRad, prefix, subrun, ymin, ymax, zmin, zmax, nEvents)


Processing Subrun 0, -600.0 < y < -300.0cm, 0.0 < z < 300.0cm...
Processing Subrun 1, -600.0 < y < -300.0cm, 300.0 < z < 600.0cm...
Processing Subrun 2, -600.0 < y < -300.0cm, 600.0 < z < 900.0cm...
Processing Subrun 3, -600.0 < y < -300.0cm, 900.0 < z < 1200.0cm...
Processing Subrun 4, -600.0 < y < -300.0cm, 1200.0 < z < 1500.0cm...
Processing Subrun 5, -600.0 < y < -300.0cm, 1500.0 < z < 1800.0cm...
Processing Subrun 6, -600.0 < y < -300.0cm, 1800.0 < z < 2100.0cm...
Processing Subrun 7, -300.0 < y < 0.0cm, 0.0 < z < 300.0cm...
Processing Subrun 8, -300.0 < y < 0.0cm, 300.0 < z < 600.0cm...
Processing Subrun 9, -300.0 < y < 0.0cm, 600.0 < z < 900.0cm...
Processing Subrun 10, -300.0 < y < 0.0cm, 900.0 < z < 1200.0cm...
Processing Subrun 11, -300.0 < y < 0.0cm, 1200.0 < z < 1500.0cm...
Processing Subrun 12, -300.0 < y < 0.0cm, 1500.0 < z < 1800.0cm...
Processing Subrun 13, -300.0 < y < 0.0cm, 1800.0 < z < 2100.0cm...
Processing Subrun 14, 0.0 < y < 300.0cm, 0.0 < z < 300.0cm...
Process